<a href="https://colab.research.google.com/github/PrahanG/MantraShastra/blob/main/MantraShastra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
file_path = "/content/data.xlsx"
xls = pd.ExcelFile(file_path)

# Load keywords from Sheet1 and remove duplicates
df_keywords = xls.parse("Sheet1")[["Keywords"]].drop_duplicates()

# Drop missing values
df_keywords = df_keywords.dropna()

# Convert keywords to lowercase and remove extra spaces
df_keywords["Processed_Keywords"] = df_keywords["Keywords"].str.lower().str.strip()

# Convert keywords to TF-IDF vectors
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df_keywords["Processed_Keywords"])

# Display results
print(df_keywords.head())
print(f"TF-IDF Matrix Shape: {tfidf_matrix.shape}")


                                            Keywords  \
0  Amba Bhavani Sharade, Devotional Song, Sri Gan...   
1  Sampoorna Bhagavad Gita Parayana, Ganapathy Sa...   
2      Solution for Financial Problems, Sundarakanda   
3  Unrevealed Secrets, Chandiyagam, Ancient Secre...   
4     Science of Bhakti yoga, Devotee and Deal Maker   

                                  Processed_Keywords  
0  amba bhavani sharade, devotional song, sri gan...  
1  sampoorna bhagavad gita parayana, ganapathy sa...  
2      solution for financial problems, sundarakanda  
3  unrevealed secrets, chandiyagam, ancient secre...  
4     science of bhakti yoga, devotee and deal maker  
TF-IDF Matrix Shape: (141, 355)


In [ ]:
from sklearn.neighbors import NearestNeighbors

# Initialize KNN model (using Cosine similarity)
knn = NearestNeighbors(n_neighbors=5, metric="cosine")
knn.fit(tfidf_matrix)

# Function to get similar keywords
def get_similar_keywords(keyword, top_n=5):
    query_vector = vectorizer.transform([keyword])  # Convert input keyword to vector
    distances, indices = knn.kneighbors(query_vector, n_neighbors=top_n)  # Find neighbors
    similar_keywords = df_keywords.iloc[indices[0]]["Processed_Keywords"].values
    return list(zip(similar_keywords, distances[0]))

# Example usage
keyword_input = "sleep"
similar_words = get_similar_keywords(keyword_input)
print("Similar Keywords:", similar_words)


Similar Keywords: [('hanuman mantra for good sleep', 0.44672407672375125), ("mantra to chant before bed for a good night's sleep", 0.6266186293154306), ('mantra for good health and happiness, kalki purana sloka', 1.0), ('wealtth, sri rama navami', 1.0), ('relief from all problems', 1.0)]


In [ ]:
import numpy as np

def mean_reciprocal_rank(results, relevant_keywords):
    ranks = []
    for i, (word, _) in enumerate(results):
        if word in relevant_keywords:
            ranks.append(1 / (i + 1))  # 1 / rank position
    return np.mean(ranks) if ranks else 0

# Example evaluation
relevant_keywords = ["relif","rest"]  # Expected similar terms
mrr_score = mean_reciprocal_rank(similar_words, relevant_keywords)
print("MRR Score:", mrr_score)


MRR Score: 0


In [ ]:
keyword_input = "sleep"
similar_words = get_similar_keywords(keyword_input)

print("Retrieved Similar Keywords:")
for word, score in similar_words:
    print(f"{word} (Similarity Score: {score})")


Retrieved Similar Keywords:
hanuman mantra for good sleep (Similarity Score: 0.44672407672375125)
mantra to chant before bed for a good night's sleep (Similarity Score: 0.6266186293154306)
mantra for good health and happiness, kalki purana sloka (Similarity Score: 1.0)
wealtth, sri rama navami (Similarity Score: 1.0)
relief from all problems (Similarity Score: 1.0)


In [ ]:
relevant_keywords = [word.lower().strip() for word in relevant_keywords]
retrieved_keywords = [word.lower().strip() for word, _ in similar_words]

print("Retrieved:", retrieved_keywords)
print("Expected:", relevant_keywords)

Retrieved: ['hanuman mantra for good sleep', "mantra to chant before bed for a good night's sleep", 'mantra for good health and happiness, kalki purana sloka', 'wealtth, sri rama navami', 'relief from all problems']
Expected: ['relif', 'rest']


In [ ]:
def mean_reciprocal_rank(results, relevant_keywords):
    ranks = []
    for i, (word, _) in enumerate(results):
        if word in relevant_keywords:  # Ensure correct string matching
            ranks.append(1 / (i + 1))
    return np.mean(ranks) if ranks else 0

# Example usage
relevant_keywords = ["health"]  # Ensure exact matches
mrr_score = mean_reciprocal_rank(similar_words, relevant_keywords)
print("MRR Score:", mrr_score)

MRR Score: 0


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# Load Excel file
file_path = "/content/data.xlsx"  # Update with your actual file path
xls = pd.ExcelFile(file_path)

# Load keywords from Sheet1 and remove duplicates & missing values
df_keywords = xls.parse("Sheet1")[["Keywords"]].drop_duplicates().dropna()

# Convert keywords to lowercase and clean text
df_keywords["Processed_Keywords"] = df_keywords["Keywords"].str.lower().str.strip()

# Convert cleaned keywords into TF-IDF vectors
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df_keywords["Processed_Keywords"])

# Initialize KNN model with Cosine Similarity
knn = NearestNeighbors(n_neighbors=5, metric="manhattan")
knn.fit(tfidf_matrix)

# Function to find similar keywords
def get_similar_keywords(keyword, top_n=5):
    query_vector = vectorizer.transform([keyword.lower().strip()])  # Convert input keyword to vector
    distances, indices = knn.kneighbors(query_vector, n_neighbors=top_n)  # Find neighbors
    similar_keywords = df_keywords.iloc[indices[0]]["Processed_Keywords"].values
    return list(zip(similar_keywords, distances[0]))

# Function to calculate Mean Reciprocal Rank (MRR)
def mean_reciprocal_rank(results, relevant_keywords):
    relevant_keywords = [word.lower().strip() for word in relevant_keywords]
    ranks = []
    for i, (word, _) in enumerate(results):
        if word in relevant_keywords:
            ranks.append(1 / (i + 1))  # 1 / rank position
    return np.mean(ranks) if ranks else 0

# Example usage
keyword_input = "health"
similar_words = get_similar_keywords(keyword_input)

# Print retrieved similar keywords
print("\n🔹 Retrieved Similar Keywords:")
for word, score in similar_words:
    print(f"{word} (Similarity Score: {score:.4f})")

# Example evaluation
relevant_keywords = ["health","wellness"]  # Expected relevant keywords
mrr_score = mean_reciprocal_rank(similar_words, relevant_keywords)
print(f"\n📌 MRR Score: {mrr_score:.4f}")



🔹 Retrieved Similar Keywords:
health (Similarity Score: 0.0000)
children's health (Similarity Score: 1.1023)
thiruppavai (Similarity Score: 2.0000)
depression (Similarity Score: 2.0000)
health, wealth, manifestation, influence (Similarity Score: 2.1819)

📌 MRR Score: 1.0000
